In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.authorities.personalName import PersonalName

In [3]:
obj = {
  "id": "n80002329",
  "label": "Machado de Assis",
  "date": "1839-1908",
  "fullerName": "Joaquim Maria Machado",
  "hasCloseExternalAuthority": ["http://id.worldcat.org/fast/48407", "http://www.wikidata.org/entity/Q311145"],
  "hasExactExternalAuthority": ["http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept"],
  "hasVariant": [
    {
      "value": "Machado de Assis, Joaquín María",
      "type": "PersonalName",
      "date": "1839-1908"
    }
  ],
}

request = PersonalName(**obj)
request

PersonalName(recordChangeDate='2023-04-06T16:31:04', id='n80002329', label='Machado de Assis', date='1839-1908', fullerName='Joaquim Maria Machado', hasCloseExternalAuthority=['http://id.worldcat.org/fast/48407', 'http://www.wikidata.org/entity/Q311145'], hasExactExternalAuthority=['http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept'], hasVariant=[Authority(value='Machado de Assis, Joaquín María', lang=None, date='1839-1908', type='PersonalName')], type='PersonalName')

In [4]:
def MakeVariant(hasVariant):
    variantTopic = """ a madsrdf:PersonalName,
                madsrdf:Variant ;
            madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                        madsrdf:elementValue "Machado de Assis, Joaquín María," ] [ a madsrdf:DateNameElement ;
                        madsrdf:elementValue "1839-1908" ] )"""
    variants = list()
    for i in hasVariant:
        if i.type == 'Topic':
            v = variantTopic.format(**i.dict())
            variants.append(v)
        elif i.type == 'ComplexSubject':
            [term1, term2] = i.value
            d = {"term1": term1, "term2":term2, "lang": i.lang}
            v = variantComplex.format(**d)
            variants.append(v)
    return variants

In [9]:
closeExternal = "madsrdf:hasCloseExternalAuthority {} ; \n"
exactExternal = "madsrdf:hasExactExternalAuthority {} ; \n"

template = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX name: <https://bibliokeia.com/authorities/name/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

INSERT DATA {{
GRAPH name:{request.id} {{
 name:{request.id} a madsrdf:Authority, madsrdf:PersonalName ;
        madsrdf:adminMetadata [ a ri:RecordInfo ; 
        ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
        ri:recordStatus "new"^^xsd:string
        ] ; 
        madsrdf:authoritativeLabel "{request.label}" ;
        madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                madsrdf:elementValue "{request.label}" ] 
                { '[ a madsrdf:DateNameElement ; madsrdf:elementValue "{}" ]'.format(request.date) if request.date else '' } ) ;
        madsrdf:fullerName [ a madsrdf:PersonalName ;
            rdfs:label "{request.fullerName}" ] ;
        { closeExternal.format(', '.join([ f'<{i}>' for i in request.hasCloseExternalAuthority])) if request.hasCloseExternalAuthority else '' }
        { exactExternal.format(', '.join([ f'<{i}>' for i in request.hasExactExternalAuthority])) if request.hasExactExternalAuthority else '' } \
}}

}}
"""
print(template)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX name: <https://bibliokeia.com/authorities/name/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

INSERT DATA {
GRAPH name:n80002329 {
 name:n80002329 a madsrdf:Authority, madsrdf:PersonalName ;
        madsrdf:adminMetadata [ a ri:RecordInfo ; 
        ri:recordChangeDate "2023-04-06T16:24:06"^^xsd:dateTime ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
        ri:recordStatus "new"^^xsd:string
        ] ; 
        madsrdf:authoritativeLabel "Machado de Assis" ;
        madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                madsrdf:elementValue "Machado de Assis" ] 
                [ a madsrdf:DateNameElement ; madsrdf:elementValue "1839-1908" ] ) ;
        madsrdf:fullerName [ a madsrdf:PersonalName ;
            r

In [2]:
g = Graph()
g.parse("http://id.loc.gov/authorities/names/n80002329.madsrdf.rdf")
g.serialize('machado.nt')

<Graph identifier=Na0e645c62e64409daecc9f5efa8537a0 (<class 'rdflib.graph.Graph'>)>

In [5]:
g = Graph()
r = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/topic/sh85017405')
g.parse(r.content)
g.serialize("subject.nt")

<Graph identifier=N54510621a4ab41878d82b0362c98c28b (<class 'rdflib.graph.Graph'>)>